In [ ]:
import argparse

from fastai.learner import Learner
from fastai.tabular.all import TabularDataLoaders
import numpy as np
from sklearn.datasets import load_iris
from torch import nn

import newron

remote_server_uri = SERVER_URI # set to your server URI
newron.set_tracking_uri(remote_server_uri)
exp_name = "FastAI_Example"
newron.set_experiment(exp_name)

In [ ]:
def get_data_loaders():
    X, y = load_iris(return_X_y=True, as_frame=True)
    y = y.astype(np.float32)
    return TabularDataLoaders.from_df(
        X.assign(target=y), cont_names=list(X.columns), y_names=y.name
    )


class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(4, 3)
        self.linear2 = nn.Linear(3, 1)

    def forward(self, _, x_cont):
        x = self.linear1(x_cont)
        return self.linear2(x)


def splitter(model):
    params = list(model.parameters())
    return [
        # weights and biases of the first linear layer
        params[:2],
        # weights and biases of the second linear layer
        params[2:],
    ]

In [ ]:
# Parse command-line arguments
lr = 0.01
epochs = 5


# Enable auto logging
newron.fastai.autolog()

# Create Learner model
learn = Learner(get_data_loaders(), Model(), loss_func=nn.MSELoss(), splitter=splitter)

# Start Newron session
with newron.start_run():
    # Train and fit with default or supplied command line arguments
    learn.fit_one_cycle(epochs, lr)

